In [17]:
import pandas as pd
import numpy as np

from datetime import timedelta

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv("data/player_price.csv")
sample_subm = pd.read_csv("data/kaggle_sample_submission.csv")

In [19]:
sample_subm["player_id"] = sample_subm["id"].apply(lambda x: int(x.split("_")[0]))
sample_subm["Date"] = sample_subm["id"].apply(lambda x: np.datetime64(x.split("_")[1]))

In [20]:
df = df[df["player_id"].isin(sample_subm["player_id"].unique())]
df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
df.drop(["timestamp", "ps_price", "player_name"], axis=1, inplace=True)

In [27]:
dataForModel = df[(df["Date"] <= df["Date"].max() - timedelta(days=0)) & 
                   (df["Date"] > df["Date"].max() - timedelta(days=14))]
dataForModel["Date"] = dataForModel["Date"].astype(str)
dataForModel = pd.pivot_table(dataForModel, values="xbox_price", 
                              index='player_id', columns='Date',
                              aggfunc=np.sum)
dataForModel.sort_index(inplace=True)

In [29]:
dataForModel.head()

Date,2017-06-09,2017-06-10,2017-06-11,2017-06-12,2017-06-13,2017-06-14,2017-06-15,2017-06-16,2017-06-17,2017-06-18,2017-06-19,2017-06-20,2017-06-21,2017-06-22
player_id,,,,,,,,,,,,,,
1,3289895,3258917,3196091,3369791,3311636,3308667,3269308,3212762,3312852,3227893,3142655,3135387,3248943,3137143
2,796667,796655,795044,786191,789865,787050,779230,784590,772174,761528,757681,747924,741715,742514
3,409258,417459,409529,401496,411600,402856,406806,404118,397944,390694,385732,379722,378681,378507
4,602222,590167,553917,553500,550667,556462,554714,555952,572346,537107,513586,528290,549743,551679
5,394579,385083,377909,379167,387667,386846,374929,383381,388962,363000,346607,349387,354543,351241


In [33]:
dataForPrediction = df[df["Date"] > df["Date"].max() - timedelta(days=14)]
dataForPrediction["Date"] = dataForPrediction["Date"].astype(str)
dataForPrediction = pd.pivot_table(dataForPrediction, values="xbox_price", 
                              index='player_id', columns='Date',
                              aggfunc=np.sum)
dataForPrediction.sort_index(inplace=True)

In [34]:
dataForPrediction.head()

Date,2017-06-09,2017-06-10,2017-06-11,2017-06-12,2017-06-13,2017-06-14,2017-06-15,2017-06-16,2017-06-17,2017-06-18,2017-06-19,2017-06-20,2017-06-21,2017-06-22
player_id,,,,,,,,,,,,,,
1,3289895,3258917,3196091,3369791,3311636,3308667,3269308,3212762,3312852,3227893,3142655,3135387,3248943,3137143
2,796667,796655,795044,786191,789865,787050,779230,784590,772174,761528,757681,747924,741715,742514
3,409258,417459,409529,401496,411600,402856,406806,404118,397944,390694,385732,379722,378681,378507
4,602222,590167,553917,553500,550667,556462,554714,555952,572346,537107,513586,528290,549743,551679
5,394579,385083,377909,379167,387667,386846,374929,383381,388962,363000,346607,349387,354543,351241


In [35]:
target_df = df[df["Date"] > df["Date"].max() - timedelta(days=7)].sort_values(["player_id"])

In [36]:
models = []
for day in target_df["Date"].unique():
    el = ElasticNet()
    el.fit(dataForModel, target_df.loc[target_df["Date"] == day, "xbox_price"])
    models.append(el)

In [37]:
for i, day in enumerate(sample_subm["Date"].unique()):
    sample_subm.loc[sample_subm["Date"] == day, "price"] = models[i].predict(dataForPrediction)

In [38]:
sample_subm[["id", "price"]].to_csv("csv/prediction_ElasticNet_2.csv", index=False)